In [ ]:
!pip install openai langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.7 MB/s eta 0:00:00


# Setup Chinook Database

In [ ]:
!wget https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip

--2023-06-16 08:09:23--  https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip
Resolving www.sqlitetutorial.net (www.sqlitetutorial.net)... 151.139.128.10
Connecting to www.sqlitetutorial.net (www.sqlitetutorial.net)|151.139.128.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 305596 (298K) [application/zip]
Saving to: ‘chinook.zip’

chinook.zip         100%[===================>] 298.43K  --.-KB/s    in 0.02s   

2023-06-16 08:09:23 (18.0 MB/s) - ‘chinook.zip’ saved [305596/305596]



In [ ]:
!unzip chinook.zip

Archive:  chinook.zip
  inflating: chinook.db              


# OpenAI Function Calling

In [ ]:
import json
import openai
import sqlite3
openai.api_key = "YOUR_API_KEY"

conn = sqlite3.connect("chinook.db")
print("Opened database successfully")
GPT_MODEL = "gpt-3.5-turbo-0613"

Opened database successfully


### Getting Table Meta

In [ ]:
def get_table_names(conn):
    """Return a list of table names."""
    table_names = []
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    for table in tables.fetchall():
        table_names.append(table[0])
    return table_names


def get_column_names(conn, table_name):
    """Return a list of column names."""
    column_names = []
    columns = conn.execute(f"PRAGMA table_info('{table_name}');").fetchall()
    for col in columns:
        column_names.append(col[1])
    return column_names


def get_database_info(conn):
    """Return a list of dicts containing the table name and columns for each table in the database."""
    table_dicts = []
    for table_name in get_table_names(conn):
        columns_names = get_column_names(conn, table_name)
        table_dicts.append({"table_name": table_name, "column_names": columns_names})
    return table_dicts

In [ ]:
database_schema_dict = get_database_info(conn)
database_schema_string = "\n".join(
    [
        f"Table: {table['table_name']}\nColumns: {', '.join(table['column_names'])}"
        for table in database_schema_dict
    ]
)

### Creating Funciton Descriptions

In [ ]:
functions = [
    {
        "name": "ask_database",
        "description": "Use this function to answer user questions about music. Output should be a fully formed SQL query.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": f"""
                            SQL query extracting info to answer the user's question.
                            SQL should be written using this database schema:
                            {database_schema_string}
                            The query should be returned in plain text, not in JSON.
                            Do not use new lines chatacthers inside the query.
                            """,
                }
            },
            "required": ["query"],
        },
    }
]

## Executing SQL queries

In [ ]:
def ask_database(conn, query):
    """Function to query SQLite database with a provided SQL query."""
    try:
        results = str(conn.execute(query).fetchall())
    except Exception as e:
        results = f"query failed with error: {e}"
    return results

def execute_function_call(message):
    if message["function_call"]["name"] == "ask_database":
        query = eval(message["function_call"]["arguments"])["query"]
        results = ask_database(conn, query)
    else:
        results = f"Error: function {message['function_call']['name']} does not exist"
    return results

## Function Calling SQL Chain Implementation
This is leterally all it takes to make a "SQLDatabaseChain" from Lanchain with function calling

In [ ]:
def openai_functions_chain(query):
  messages = []
  messages.append({"role": "system", "content": "Answer user questions by generating SQL queries against the Chinook Music Database."})
  messages.append({"role": "user", "content": query})
  while True:
    assistant_message = openai.ChatCompletion.create(
        temperature=0,
        model=GPT_MODEL,
        messages=messages,
        functions=functions,
        function_call="auto",
    )["choices"][0]["message"]
    messages.append(assistant_message)

    if assistant_message.get("function_call"):
      print("Executing function: ", assistant_message["function_call"])
      results = execute_function_call(assistant_message)
      messages.append({"role": "function", "name": assistant_message["function_call"]["name"], "content": results})
    else:
      break

  return assistant_message['content']

# Langchain Implementation

In [ ]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentExecutor
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from langchain.agents import AgentType
from langchain import SQLDatabaseChain

In [ ]:
db = SQLDatabase.from_uri("sqlite:///chinook.db")
llm = ChatOpenAI(temperature=0, openai_api_key=openai.api_key)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, top_k=3)

# Langchain vs Function Calling

### **Query:** Identify the most popular genre among customers based on the number of tracks sold in that genre. (Easy)
Both were able to answer.

In [ ]:
query = "Identify the most popular genre among customers based on the number of tracks sold in that genre."

In [ ]:
print("Response: ", db_chain.run(query))



> Entering new  chain...
Identify the most popular genre among customers based on the number of tracks sold in that genre.
SQLQuery:SELECT genres.Name, SUM(invoice_items.Quantity) AS total_sold
FROM genres
JOIN tracks ON genres.GenreId = tracks.GenreId
JOIN invoice_items ON tracks.TrackId = invoice_items.TrackId
GROUP BY genres.Name
ORDER BY total_sold DESC
LIMIT 1
SQLResult: [('Rock', 835)]
Answer:The most popular genre among customers based on the number of tracks sold in that genre is Rock, with a total of 835 tracks sold.
> Finished chain.
Response:  The most popular genre among customers based on the number of tracks sold in that genre is Rock, with a total of 835 tracks sold.


In [ ]:
print("Response: ", openai_functions_chain(query))

Executing function:  {
  "name": "ask_database",
  "arguments": "{\n  \"query\": \"SELECT g.Name AS Genre, COUNT(*) AS TracksSold FROM genres g JOIN tracks t ON g.GenreId = t.GenreId JOIN invoice_items ii ON t.TrackId = ii.TrackId GROUP BY g.GenreId ORDER BY TracksSold DESC LIMIT 1;\"\n}"
}
Response:  The most popular genre among customers based on the number of tracks sold is Rock, with a total of 835 tracks sold.


### **Query:** Find the customer who has spent the most in each genre and display their total spending for each genre. (Medium)
Langchain implemetation tells you to look at the query instead of providing clear interpretation of the result.

In [ ]:
query = "Find the customer who has spent the most in each genre and display their total spending for each genre."

In [ ]:
print("Response: ", db_chain.run(query))



> Entering new  chain...
Find the customer who has spent the most in each genre and display their total spending for each genre.
SQLQuery:SELECT g.Name AS Genre, c.FirstName || ' ' || c.LastName AS Customer, SUM(i.Total) AS TotalSpent
FROM invoices i
JOIN customers c ON i.CustomerId = c.CustomerId
JOIN invoice_items ii ON i.InvoiceId = ii.InvoiceId
JOIN tracks t ON ii.TrackId = t.TrackId
JOIN genres g ON t.GenreId = g.GenreId
GROUP BY g.Name, c.CustomerId
HAVING SUM(i.Total) = (
  SELECT MAX(TotalSpent)
  FROM (
    SELECT g.Name AS Genre, c.FirstName || ' ' || c.LastName AS Customer, SUM(i.Total) AS TotalSpent
    FROM invoices i
    JOIN customers c ON i.CustomerId = c.CustomerId
    JOIN invoice_items ii ON i.InvoiceId = ii.InvoiceId
    JOIN tracks t ON ii.TrackId = t.TrackId
    JOIN genres g ON t.GenreId = g.GenreId
    GROUP BY g.Name, c.CustomerId
  ) AS subquery
  WHERE subquery.Genre = g.Name
)
ORDER BY g.Name ASC
SQLResult: [('Alternative', 'Frank Ralston', 79.3), ('Altern

In [ ]:
print("Response: ", openai_functions_chain(query))

Executing function:  {
  "name": "ask_database",
  "arguments": "{\n  \"query\": \"SELECT g.Name AS Genre, c.FirstName || ' ' || c.LastName AS Customer, SUM(i.Total) AS TotalSpending FROM genres g JOIN tracks t ON g.GenreId = t.GenreId JOIN invoice_items ii ON t.TrackId = ii.TrackId JOIN invoices i ON ii.InvoiceId = i.InvoiceId JOIN customers c ON i.CustomerId = c.CustomerId GROUP BY g.Name, c.CustomerId HAVING SUM(i.Total) = (SELECT MAX(TotalSpending) FROM (SELECT g.Name AS Genre, c.FirstName || ' ' || c.LastName AS Customer, SUM(i.Total) AS TotalSpending FROM genres g JOIN tracks t ON g.GenreId = t.GenreId JOIN invoice_items ii ON t.TrackId = ii.TrackId JOIN invoices i ON ii.InvoiceId = i.InvoiceId JOIN customers c ON i.CustomerId = c.CustomerId GROUP BY g.Name, c.CustomerId) AS temp WHERE temp.Genre = g.Name)\"\n}"
}
Response:  The customer who has spent the most in each genre and their total spending for each genre are as follows:

- Genre: Alternative, Customer: Frank Ralston, Tot

### **Query:** Identify the artist with the highest revenue, excluding compilations, and display their top-selling track. (Hard)
Langhcain implemetation fails, funcion calling wins.

In [ ]:
query = "Identify the artist with the highest revenue, excluding compilations, and display their top-selling track."

In [ ]:
print("Response: ", db_chain.run(query))



> Entering new  chain...
Identify the artist with the highest revenue, excluding compilations, and display their top-selling track.
SQLQuery:SELECT artists.Name, tracks.Name, SUM(invoice_items.Quantity * invoice_items.UnitPrice) AS revenue
FROM tracks
JOIN albums ON tracks.AlbumId = albums.AlbumId
JOIN artists ON albums.ArtistId = artists.ArtistId
JOIN invoice_items ON tracks.TrackId = invoice_items.TrackId
JOIN invoices ON invoice_items.InvoiceId = invoices.InvoiceId
WHERE genres.Name != "Compilation"
GROUP BY artists.ArtistId
ORDER BY revenue DESC
LIMIT 1;

OperationalError: ignored

In [ ]:
print("Response: ", openai_functions_chain(query))

Executing function:  {
  "name": "ask_database",
  "arguments": "{\n  \"query\": \"WITH customer_spending AS (SELECT c.CustomerId, g.Name AS Genre, SUM(i.Total) AS TotalSpending FROM customers c JOIN invoices i ON c.CustomerId = i.CustomerId JOIN invoice_items ii ON i.InvoiceId = ii.InvoiceId JOIN tracks t ON ii.TrackId = t.TrackId JOIN genres g ON t.GenreId = g.GenreId GROUP BY c.CustomerId, g.Name) SELECT cs.CustomerId, c.FirstName || ' ' || c.LastName AS CustomerName, cs.Genre, cs.TotalSpending FROM customer_spending cs JOIN customers c ON cs.CustomerId = c.CustomerId WHERE cs.TotalSpending = (SELECT MAX(TotalSpending) FROM customer_spending WHERE Genre = cs.Genre) ORDER BY cs.Genre\"\n}"
}
Response:  The query results in the following output:

| CustomerId | CustomerName       | Genre              | TotalSpending |
|------------|--------------------|--------------------|---------------|
| 24         | Frank Ralston      | Alternative        | 79.3          |
| 8          | Daan Pee

# Test yourself

In [ ]:
query = "Identify the most popular genre among customers based on the number of tracks sold in that genre." #@param {type:"string"}

In [ ]:
print("Response: ", db_chain.run(query))

In [ ]:
print("Response: ", openai_functions_chain(query))